In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.5/203.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [1]:
!pip install sentence-transformers

In [2]:
pip install langchain-groq

In [3]:
pip install langchain_huggingface

  Using cached langchain_huggingface-0.0.3-py3-none-any.whl (17 kB)


In [4]:
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools  import tool
from crewai import Crew
from crewai import Task
from crewai import Agent
from google.colab import userdata

In [5]:
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [13]:
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],
    model_name="llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000,
)

In [14]:
rag_tool = PDFSearchTool(pdf='Telecom Proposal.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
rag_tool.run("How to create a proposal for a telecommunications company")

Using Tool: Search a PDF's content


'Relevant Content:\npartnership, you almost always need something as consolidating as a business proposal to convince the other party. The idea of writing a comprehensive business proposal may sound quite daunting to you if you’ve never done anything similar before. But you don’t need to worry because we’re here to help you with that through our comprehensive guide to writing a winning business proposal for the telecom industry. 1. BUSINESS PROPOSAL TEMPLATE FOR THE TELECOMS INDUSTRY You need to understand two things before actually getting to the business proposal template itself: 1. An excellent business proposal should always be customer-centric, customized to the needs and requirements of your potential customer or partner.\n\nI. BUSINESS PROPOSALS FOR THE TELECOM INDUSTRY: A COMPLETE GUIDE The telecommunications industry lies at the heart of today’s fast-paced digital world. That’s why it is among the fastest-growing sectors of the world right now. A wide variety of businesses and

In [15]:
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
web_search_tool = TavilySearchResults(k=3)

In [16]:
web_search_tool.run("How to create a proposal for a telecommunications company")

[{'url': 'https://goprospero.com/blog/business-proposals-telecom-industry/',
  'content': "Business Proposal Template for the Telecommunications Industry. Executive Summary: A generic business overview, industry overview, the problem, and how you'll solve it. Product or Service Description: A general explanation of what your product does and its benefits for your prospective client. Market Analysis: An overall analysis of the market, focusing specifically on the conditions that ..."},
 {'url': 'https://www.proposalkit.com/htm/how-to-write-a-telecommunications-proposal.htm',
  'content': 'Keep this short - just introduce yourself, explain why you are sending your proposal at this time, state what you would like the reader to do after considering the proposal (call you, sign the enclosed contract, set up a meeting, etc.), and provide all your contact information. The next page should be a Title Page for your proposal.'},
 {'url': 'https://www.bplans.com/telecommunications-business-plan/'

In [17]:
@tool
def router_tool(question):
  """Router Function"""
  if 'proposal for a telecommunications company' in question:
    return 'vectorstore'
  else:
    return 'web_search'

In [27]:
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()
# Define your agents with roles and goals
researcher = Agent(
  role='Market Research Analyst',
  goal='To gather, analyze, and interpret data regarding market trends, customer behavior, and competitive landscape to provide insights that will guide strategic business decisions',
  backstory="""You are an experienced market research analyst with a strong
  background in telecommunications. Your role is to dive deep into market data,
  understand the nuances of customer preferences, and identify emerging trends.
  By leveraging advanced data analytics tools and methodologies, you provide comprehensive
  market insights that help the company stay ahead of the competition and adapt to changing market dynamics.""",
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [28]:
tech = Agent(
  role='Technology Expert',
  goal='To evaluate and implement cutting-edge technologies that enhance the company’s telecommunications infrastructure, improve service delivery, and drive innovation.',
  backstory="""You are a seasoned technology expert with extensive experience in telecommunications.
  Your expertise spans across network infrastructure, emerging technologies such as 5G, IoT, and AI.
   Your mission is to identify the best technological solutions that align with the company’s strategic goals.
   By continuously exploring new advancements and integrating them into the company’s operations, you ensure that
   the company remains a leader in technological innovation and service excellence.""",
  verbose=True,
  allow_delegation=True,
  llm=llm,
)

In [29]:
finance = Agent(
  role='Financial Expert',
  goal='To provide financial analysis and strategic recommendations that optimize the company’s financial performance and support sustainable growth.',
  backstory="""You are a financial expert with a deep understanding of the telecommunications industry.
  Your role involves analyzing financial data, developing financial models, and evaluating investment opportunities.
  By providing accurate and insightful financial advice, you help the company make informed decisions that enhance
  profitability, manage risks, and achieve long-term financial stability.""",
  verbose=True,
  allow_delegation=True,
  llm=llm,
)

In [30]:
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Evaluate the implementation feasibility of 5G and IoT technologies in the company's network infrastructure.
  Assess potential benefits and challenges.""",
  expected_output="Feasibility assessment report with benefits and challenges",
  agent=tech
)
task3 = Task(
  description="""Analyze the financial implications of adopting new AI-driven technologies.
  Develop a cost-benefit analysis and forecast potential ROI.""",
  expected_output="Detailed cost-benefit analysis report with ROI forecast",
  agent=finance
)
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, tech, finance],
  tasks=[task1, task2, task3],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

 [DEBUG]: == Working Agent: Market Research Analyst
 [INFO]: == Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024. 
  Identify key trends, breakthrough technologies, and potential industry impacts.


> Entering new CrewAgentExecutor chain...
Thought: I'm excited to dive into the latest advancements in AI in 2024! With my expertise in market research and telecommunications, I'm confident that I can provide a comprehensive analysis of the key trends, breakthrough technologies, and potential industry impacts.

Final Answer:

**Comprehensive Analysis Report: AI Advancements in 2024**

**Key Trends:**

• **Explainable AI (XAI):** The increasing demand for transparency and accountability in AI decision-making has led to the development of XAI, which enables users to understand the reasoning behind AI-driven decisions.

• **Edge AI:** The proliferation of IoT devices and the need for real-time processing have driven the growth of Edge AI, which enables A

In [31]:
print(result)

**Executive Summary:**

The adoption of AI-driven technologies in telecommunications is a strategic imperative for companies seeking to improve customer experience, increase efficiency, and enhance network performance. This report provides a comprehensive analysis of the current state of AI-driven technologies in telecommunications, including key trends, breakthroughs, and potential applications. The report also includes market research and analysis on the adoption rates and potential ROI of these technologies.

**Introduction:**

The telecommunications industry is undergoing a significant transformation driven by the adoption of AI-driven technologies. AI is being used to optimize network performance, predict and prevent outages, and improve customer experience. This report provides a comprehensive analysis of the current state of AI-driven technologies in telecommunications, including key trends, breakthroughs, and potential applications.

**Key Trends:**

1. **Artificial Intelligenc